In [2]:
import polars as pl
import altair

In [3]:
stations = pl.read_csv("data/bluebike_stations_geolocated.csv", columns=["name", "City"])
cambridge_stations = stations.filter(pl.col("City") == "Cambridge")
display(cambridge_stations)
pl.Config.set_fmt_str_lengths(200)
cambridge_station_ids = {s for s in cambridge_stations.to_dict()["name"]}

name,City
str,str
"""Harvard Univer…","""Cambridge"""
"""Ames St at Mai…","""Cambridge"""
"""Kendall Street…","""Cambridge"""
"""MIT Pacific St…","""Cambridge"""
"""Central Square…","""Cambridge"""
"""One Broadway /…","""Cambridge"""
"""Harvard Square…","""Cambridge"""
"""CambridgeSide …","""Cambridge"""
"""Cambridge St -…","""Cambridge"""


In [4]:
rides = pl.read_csv("data/20*.csv", columns=["starttime", "start station id", "end station id", "start station name", "end station name"]).with_columns(pl.col("starttime").str.slice(0, 10).str.strptime(pl.Date, fmt="%Y-%m-%d", strict=False)).sort("starttime")

In [5]:
rides.filter(pl.col("starttime").dt.year() == 2022)

starttime,start station id,end station id,start station name,end station name
date,i64,i64,str,str
2022-01-01,178,74,"""MIT Pacific St at Purrington St""","""Harvard Square at Mass Ave/ Dunster"""
2022-01-01,189,178,"""Kendall T""","""MIT Pacific St at Purrington St"""
2022-01-01,94,356,"""Main St at Austin St""","""Charlestown Navy Yard"""
2022-01-01,94,356,"""Main St at Austin St""","""Charlestown Navy Yard"""
2022-01-01,19,41,"""Park Dr at Buswell St""","""Packard's Corner - Commonwealth Ave at Brighton Ave"""
2022-01-01,107,68,"""Ames St at Main St""","""Central Square at Mass Ave / Essex St"""
2022-01-01,36,36,"""Copley Square - Dartmouth St at Boylston St""","""Copley Square - Dartmouth St at Boylston St"""
2022-01-01,58,58,"""Mugar Way at Beacon St""","""Mugar Way at Beacon St"""
2022-01-01,60,363,"""Charles Circle - Charles St at Cambridge St""","""Harrison Ave at Mullins Way"""


In [6]:
cambridge_rides = rides.filter(pl.col("start station name").is_in(cambridge_station_ids) | pl.col("end station name").is_in(cambridge_station_ids))
cambridge_rides

starttime,start station id,end station id,start station name,end station name
date,i64,i64,str,str
2016-01-01,36,67,"""Boston Public Library - 700 Boylston St.""","""MIT at Mass Ave / Amherst St"""
2016-01-01,110,88,"""Harvard University Gund Hall at Quincy St / Kirkland S""","""Inman Square at Vellucci Plaza / Hampshire St"""
2016-01-01,107,176,"""Ames St at Main St""","""Lesley University"""
2016-01-01,141,90,"""Kendall Street""","""Lechmere Station at Cambridge St / First St"""
2016-01-01,178,80,"""MIT Pacific St at Purrington St""","""MIT Stata Center at Vassar St / Main St"""
2016-01-01,68,178,"""Central Square at Mass Ave / Essex St""","""MIT Pacific St at Purrington St"""
2016-01-01,72,178,"""One Broadway / Kendall Sq at Main St / 3rd St""","""MIT Pacific St at Purrington St"""
2016-01-01,68,177,"""Central Square at Mass Ave / Essex St""","""University Park"""
2016-01-01,68,177,"""Central Square at Mass Ave / Essex St""","""University Park"""


In [7]:
def by_month(df):
    return df.groupby_dynamic("starttime", every="1mo").agg(pl.count())

def by_year(df):
    return df.groupby_dynamic("starttime", every="1y").agg(pl.count())

monthly = by_month(cambridge_rides)
display(monthly.select([pl.col("count").sum()]))
yearly = by_year(cambridge_rides)
display(yearly.select([pl.col("count").sum()]))
display(monthly.filter(pl.col("starttime").dt.year() >= 2022))
display(yearly)

count
u32
7482990


count
u32
7482990


starttime,count
date,u32
2022-01-01,43555
2022-02-01,63646
2022-03-01,93223
2022-04-01,137045
2022-05-01,163078
2022-06-01,169680
2022-07-01,191386
2022-08-01,207982
2022-09-01,254331


starttime,count
date,u32
2016-01-01,588701
2017-01-01,651458
2018-01-01,878302
2019-01-01,1192178
2020-01-01,831471
2021-01-01,1327930
2022-01-01,1749436
2023-01-01,263514


In [8]:
from datetime import datetime

broadway = pl.read_csv("data/broadway.csv.gz", dtypes={"Day": pl.Categorical}, try_parse_dates=True).with_columns(pl.col("Date").str.strptime(pl.Date, fmt="%m/%d/%Y")).sort("Date")
# Filter out partial months
broadway = broadway.filter((pl.col("Date") >= datetime(2016, 1, 1)) & (pl.col("Date") < datetime(2023, 1, 1)))
broadway

DateTime,Day,Date,Time,Total,Westbound,Eastbound
str,cat,date,time,i64,i64,i64
"""01/01/2016 12:00:00 AM""","""Friday""",2016-01-01,00:00:00,1,1,0
"""01/01/2016 12:15:00 AM""","""Friday""",2016-01-01,00:15:00,1,1,0
"""01/01/2016 12:30:00 AM""","""Friday""",2016-01-01,00:30:00,1,1,0
"""01/01/2016 12:45:00 AM""","""Friday""",2016-01-01,00:45:00,0,0,0
"""01/01/2016 01:00:00 AM""","""Friday""",2016-01-01,01:00:00,0,0,0
"""01/01/2016 01:15:00 AM""","""Friday""",2016-01-01,01:15:00,0,0,0
"""01/01/2016 01:30:00 AM""","""Friday""",2016-01-01,01:30:00,0,0,0
"""01/01/2016 01:45:00 AM""","""Friday""",2016-01-01,01:45:00,1,0,1
"""01/01/2016 02:00:00 AM""","""Friday""",2016-01-01,02:00:00,1,1,0


In [9]:
broadway_yearly = broadway.groupby(pl.col("Date").dt.year()).agg(pl.col("Total").sum().alias("Broadway count")).sort("Date")
display(broadway_yearly)
bluebikes_plus_broadway = yearly.rename({"count": "BlueBikes to/from Cambridge"}).filter(pl.col("starttime").dt.year() < 2023).rename({"starttime": "Year"}).with_columns(broadway_yearly.get_column("Broadway count"))
bluebikes_plus_broadway
alt = altair
altair.Chart(bluebikes_plus_broadway.filter(pl.col("Year") >= 2016).to_pandas()).mark_line().encode(
    x=alt.X("Year"),
    y=alt.Y(altair.repeat('layer'), type="quantitative").title("Rides"),
    color=altair.ColorDatum(altair.repeat('layer'))
).repeat(layer=["BlueBikes to/from Cambridge", "Broadway count"]).properties(title="Annual rides")

Date,Broadway count
i32,i64
2016,383844
2017,336919
2018,315111
2019,428411
2020,271296
2021,292089
2022,324530


alt.RepeatChart(...)

In [10]:
display(yearly.select(pl.col("count").sum()))
to_2022 = yearly.filter(pl.col("starttime").dt.year() < 2023).rename({"starttime": "Year", "count": "# of rides"})
altair.Chart(to_2022.to_pandas()).mark_line().encode(x="Year:T", y=f"# of rides:Q").properties(title="Yearly BlueBike rides starting or ending in Cambridge")

count
u32
7482990


alt.Chart(...)

In [11]:

def linechart(df, title, y_column="count"):
    return altair.Chart(df.to_pandas()).mark_line().encode(x="starttime:T", y=f"{y_column}:Q").properties(title=title)

display(linechart(monthly, 'Monthly Bluebikes rides starting/ending in Cambridge'))
display(linechart(yearly.filter(pl.col("starttime").dt.year() < 2023), 'Yearly Bluebikes rides starting/ending in Cambridge (2016-2022)'))
display(linechart(broadway.groupby_dynamic("Date", every="1y").agg(pl.count()), 'Broadway EcoTotem'))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

In [12]:
monthly2 = monthly.select([pl.col("starttime").dt.month().alias("Month"), pl.col("starttime").dt.year().alias("Year"), pl.col("count").alias("Count")])
monthly2

Month,Year,Count
u32,i32,u32
1,2016,12031
2,2016,14631
3,2016,26227
4,2016,44246
5,2016,57531
6,2016,70931
7,2016,73552
8,2016,74497
9,2016,76185


In [13]:
month_per_year = monthly2.pivot(values="Count", index="Month", columns="Year")
month_per_year

/tmp/ipykernel_397047/2079401287.py:1: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  month_per_year = monthly2.pivot(values="Count", index="Month", columns="Year")


Month,2016,2017,2018,2019,2020,2021,2022,2023
u32,u32,u32,u32,u32,u32,u32,u32,u32
1,12031,17181,23533,37108,65306,34562,43555,72857
2,14631,17221,35994,44724,71408,30185,63646,85216
3,26227,22636,35682,55133,53050,64627,93223,105441
4,44246,50419,52708,83450,16506,81994,137045,null
5,57531,60602,86049,105137,41073,116425,163078,null
6,70931,73351,95606,122512,66621,123875,169680,null
7,73552,79994,115805,142630,92341,126275,191386,null
8,74497,85348,112517,146264,103762,149732,207982,null
9,76185,79270,115410,165361,117601,204170,254331,null


In [14]:
alt = altair
altair.Chart(month_per_year.to_pandas()).mark_line().encode(
    x=alt.X("Month"),
    y=alt.Y(altair.repeat('layer'), type="quantitative").title("Monthly rides"),
    color=altair.ColorDatum(altair.repeat('layer'), type="ordinal")
).repeat(layer=[str(y) for y in range(2019, 2024)]).properties(title="Monthly BlueBikes rides to/from Cambridge")


alt.RepeatChart(...)

In [15]:
porter = cambridge_rides.filter((pl.col("start station name") == "Porter Square Station") | (pl.col("end station name") == "Porter Square Station"))
porter_monthly = by_month(porter)
display(porter_monthly)

starttime,count
date,u32
2016-01-01,634
2016-02-01,644
2016-03-01,983
2016-04-01,1599
2016-05-01,2108
2016-06-01,2485
2016-07-01,2486
2016-08-01,2353
2016-09-01,2392


In [16]:
year_over_year = porter_monthly.select(pl.col("starttime"), pl.col("count") / pl.col("count").shift(12)).filter(pl.col("starttime").dt.month().is_in([7, 8, 9, 10, 11]))
display(linechart(year_over_year, "Year-over-Year growth, Porter Sq. BlueBikes Station"))
year_over_year = by_year(porter).select(pl.col("starttime"), pl.col("count") / pl.col("count").shift(1))
linechart(year_over_year, "Year-over-Year growth, Porter Sq. BlueBikes Station")

alt.Chart(...)

alt.Chart(...)

In [17]:
monthly2 = monthly.sort("starttime").with_columns(porter_monthly.sort("starttime").get_column("count").alias("Porter"))
display(monthly2)
yearly2 = monthly2.groupby(pl.col("starttime").dt.year()).agg([pl.col("count").sum(), pl.col("Porter").sum()]).sort("starttime")
display(yearly2)
display(yearly2.select([pl.col("starttime"), pl.col("count") / 588701, pl.col("Porter") / 19961]))
linechart(monthly2.select([pl.col("starttime"), ((pl.col("Porter") / pl.col("count")) * 100).alias("Porter %")]), "", "Porter %")

starttime,count,Porter
date,u32,u32
2016-01-01,12031,634
2016-02-01,14631,644
2016-03-01,26227,983
2016-04-01,44246,1599
2016-05-01,57531,2108
2016-06-01,70931,2485
2016-07-01,73552,2486
2016-08-01,74497,2353
2016-09-01,76185,2392


starttime,count,Porter
i32,u32,u32
2016,588701,19961
2017,651458,18193
2018,878302,24107
2019,1192178,30325
2020,831471,18395
2021,1327930,25245
2022,1749436,35131
2023,263514,5980


starttime,count,Porter
i32,f64,f64
2016,1.0,1.0
2017,1.106603,0.911427
2018,1.491932,1.207705
2019,2.025099,1.519212
2020,1.412383,0.921547
2021,2.255695,1.264716
2022,2.971689,1.759982
2023,0.447619,0.299584


alt.Chart(...)